In [1]:
# importing necessary libraries
import pandas as pd
import numpy as np

# reading cab rides data into dataframe
rides_df = pd.read_csv('dataOriginal/cab_rides.csv')

# reading weather data into dataframe
weather_df = pd.read_csv('dataOriginal/weather.csv')

In [ ]:
# Converting Unix timestamp to Datetime, truncated to nearest hour
rides_df = rides_df.copy()
rides_df['time_stamp'] = pd.to_datetime(rides_df['time_stamp'], unit='ms').dt.floor('30min')
weather_df = weather_df.copy()
weather_df['time_stamp'] = pd.to_datetime(rides_df['time_stamp'], unit='ms').dt.round('30min')

In [5]:
print(rides_df.shape)
print(weather_df.shape )

(693071, 10)
(6276, 8)


In [6]:
weather_df = weather_df.drop_duplicates(subset=['time_stamp', 'location'], keep='first')
print(weather_df.shape )

(4147, 8)


In [7]:
# removing Null Values from rides dataframe 
rides_df = rides_df.dropna()

# Remove product id from rides df
rides_df = rides_df.drop('product_id', axis = 1)

# Converting id from 30 characters -> 0, 1, 2, etc..
rides_df['id'] = range(len(rides_df))


# converting null values in (rain) column to 0 (No rain)
weather_df['rain'] = weather_df['rain'].fillna(0)

In [8]:
# splitting dataframes into Uber and Lyft
uber_df = rides_df[rides_df['cab_type'] == 'Uber']
lyft_df = rides_df[rides_df['cab_type'] == 'Lyft']

In [9]:
print(uber_df.shape)
print(lyft_df.shape)

(330568, 9)
(307408, 9)


In [10]:
from google.cloud import bigquery
from google.api_core.exceptions import NotFound

# Initialize BigQuery client
client = bigquery.Client.from_service_account_json("secrets/serviceKey.json")

# Define the dataset reference
project_id = "idmpproject-441123"
dataset_id = "uberFareEstimation"
dataset_ref = f"{project_id}.{dataset_id}"

# List all tables in the dataset
tablesList = []
try:
    tables = client.list_tables(dataset_ref)
    print(f"Tables in dataset {dataset_id}:")
    for table in tables:
        tablesList.append(table.table_id)
        print(table.table_id)
except NotFound:
    print(f"Dataset {dataset_id} does not exist.")


Tables in dataset uberFareEstimation:
lyft_data
uber_data
weather_data


In [11]:
rides_schema = [
    bigquery.SchemaField("distance", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("cab_type", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("time_stamp", "TIMESTAMP", mode="NULLABLE"),
    bigquery.SchemaField("destination", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("source", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("price", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("surge_multiplier", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("id", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("name", "STRING", mode="NULLABLE")
]

weather_schema = [
    bigquery.SchemaField("temp", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("location", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("clouds", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("pressure", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("rain", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("time_stamp", "TIMESTAMP", mode="NULLABLE"),
    bigquery.SchemaField("humidity", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("wind", "FLOAT", mode="NULLABLE")
]

In [12]:
if 'uber_data' not in tablesList:
    table_id = "uber_data"
    table_ref = f"{project_id}.{dataset_id}.{table_id}"
    table_object = bigquery.Table(table_ref, schema=rides_schema)
    try:
        table = client.create_table(table_object)
        print(f"Created empty table {table_id} in dataset {dataset_id}.")
    except Exception as e:
        print(f"Error creating table: {e}")


if 'lyft_data' not in tablesList:
    table_id = "lyft_data"
    table_ref = f"{project_id}.{dataset_id}.{table_id}"
    table_object = bigquery.Table(table_ref, schema=rides_schema)
    try:
        table = client.create_table(table_object)
        print(f"Created empty table {table_id} in dataset {dataset_id}.")
    except Exception as e:
        print(f"Error creating table: {e}")
        
        
if 'weather_data' not in tablesList:
    table_id = "weather_data"
    table_ref = f"{project_id}.{dataset_id}.{table_id}"
    table_object = bigquery.Table(table_ref, schema=rides_schema)
    try:
        table = client.create_table(table_object)
        print(f"Created empty table {table_id} in dataset {dataset_id}.")
    except Exception as e:
        print(f"Error creating table: {e}")



In [17]:
def overWriteDataToTable(dataFrameName, table_ref):
    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_TRUNCATE",  # Overwrite the data to the table
    )
    job = client.load_table_from_dataframe(dataFrameName, table_ref, job_config=job_config)

    # Wait for the job to complete
    job.result()

    print(f"Over written {len(dataFrameName)} rows to the table {table_id} in dataset {dataset_id}.")

In [21]:
while True:
    user_input = input("Which table do you want to overwrite the data?\n 1. lyft_data\n 2. uber_data\n 3. weather_data")
    if user_input in ['1', '2', '3']:  # Check if the input is valid
        print(f"Valid input received: {user_input}")
        break  # Exit the loop if the input is valid
    else:
        print("Invalid input. Please try again.")


Valid input received: 3


In [22]:
if user_input == '1': 
    table_id = "lyft_data"
    table_ref = f"{project_id}.{dataset_id}.{table_id}"
    authorization = input("Are you sure to over write data? Type 'OVERWRITE' to proceed.")
    if authorization == 'OVERWRITE':
        overWriteDataToTable(lyft_df, table_ref)
    else:
        print("Action aborted")
if user_input == '2':
    table_id = "uber_data"
    table_ref = f"{project_id}.{dataset_id}.{table_id}"
    authorization = input("Are you sure to over write data? Type 'OVERWRITE' to proceed.")
    if authorization == 'OVERWRITE':
        overWriteDataToTable(uber_df, table_ref)
    else:
        print("Action aborted")
if user_input == '3':
    table_id = 'weather_data'
    table_ref = f"{project_id}.{dataset_id}.{table_id}"
    authorization = input("Are you sure to over write data? Type 'OVERWRITE' to proceed.")
    if authorization == 'OVERWRITE':
        overWriteDataToTable(weather_df, table_ref)
    else:
        print("Action aborted")

Over written 4147 rows to the table weather_data in dataset uberFareEstimation.


In [ ]:
print("hello")